In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import pandas_profiling as pp
import seaborn as sns

In [ ]:
data = pd.read_csv('../input/students-performance-in-exams/StudentsPerformance.csv')

In [ ]:
data.head(10)

In [ ]:
data.info()

I must change the column names.

In [ ]:
data.rename(columns = {'parental level of education':'parentEducation', 
                       'test preparation course':'testCourse', 
                       'math score':'mathScore',
                       'reading score':'readingScore', 
                       'writing score':'writingScore',
                        'race/ethnicity': 'race'}, inplace=True)

In [ ]:
pp.ProfileReport(data)

Firstly we are looking for descriptive statistics with pandas profil report. There ise no null data. Correlations between scores does not surprise me. We will learn more about grouping operations and visualization operations. I want to get the average score of each student for a few operations.I don't think this will be a problem since there is a correlation between the scores.


In [ ]:
data['score'] = ((data['mathScore']+ data['readingScore'] + data['writingScore'])/ 3)

In [ ]:
data.head(5)

In [ ]:
data.groupby(['race', 'parentEducation'])['score'].mean()

In [ ]:
plt.figure(figsize=(10,5))
data.gender.value_counts().plot(kind='barh',color=['salmon', 'lightgreen'])
plt.xlabel('Count', fontsize=20)
plt.ylabel('Gender', fontsize=20)
y = data.gender.value_counts()

for index, count in enumerate(y):
    plt.text(count, index,
             str(count))

In [ ]:
plt.figure(figsize=(10,20))
plt.subplot(3,1,1)
sns.distplot(data['mathScore'], color='red');
plt.subplot(3,1,2)
sns.distplot(data['writingScore'],color='black')
plt.subplot(3,1,3)
sns.distplot(data['readingScore'], color='blue')

In [ ]:
plt.figure(figsize=(10,15))
plt.subplot(3,1,1)
sns.scatterplot(x='mathScore', y="readingScore", hue='gender', data=data)
plt.subplot(3,1,2)
sns.scatterplot(x='mathScore', y='writingScore', hue='gender', data=data)
plt.subplot(3,1,3)
sns.scatterplot(x='readingScore', y='writingScore', hue='gender', data=data)

In [ ]:
group_data = data.groupby('race')["mathScore", "readingScore", "writingScore"].mean().round()
group_data.plot(kind='bar', figsize=(14, 7), color=['LightSalmon','Crimson','DarkRed'],width=0.2)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(data.corr(), annot=True, cmap='twilight_shifted')

In [ ]:
data['race'] = data.race.astype('category')
data['gender'] = data.gender.astype('category')
data['parentEducation'] = data.parentEducation.astype('category')
data['testCourse'] = data.testCourse.astype('category')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
data["race"] = le.fit_transform(data["race"])
data["gender"] = le.fit_transform(data["gender"])
data["parentEducation"] = le.fit_transform(data["parentEducation"])
data["testCourse"] = le.fit_transform(data["testCourse"])
data["lunch"] = le.fit_transform(data["lunch"])

In [ ]:
data.head()

In [ ]:
score = data["score"]

In [ ]:
data = data.drop(["score"], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, score, test_size = 0.30)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rcr = RandomForestRegressor(random_state = 42)

In [ ]:
rcr.fit(x_train,y_train)

In [ ]:
y_pred= rcr.predict(x_test)

In [ ]:
np.sqrt(((y_pred - y_test) ** 2).mean())

In [ ]:
print("Train Accuracy:",rcr.score(x_train, y_train))
print("Test Accuracy:",rcr.score(x_test, y_test))